# Quick start

*EPAM Syngen* is an unsupervised tabular data generation tool based on a variational autoencoder (VAE). 
It supports common tabular datatypes (floats, integers, datetime, text, categorical, binary) and can generate linked tables that sharing keys using the simple statistical approach. 
The SDK exposes simple programmatic entry points for training, inference, report generation, loading and saving data in supported formats - *CSV*, *Avro* and *Excel* format. The data should be located locally and be in UTF-8 encoding.

This notebook demonstrates the SDK usage. Install the package and then you can call the main SDK class `Syngen` to run training, inference or generation of reports, and the class `DataIO` to load and save the data in supported formats.

Python *3.10* or *3.11* is required to run the library. The library is tested on Linux and Windows operating systems.

# Installation

Please, install the library *syngen* (from Pypi):

In [1]:
!pip install  --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ --use-pep517 --no-cache-dir syngen==0.10.33rc3

Looking in indexes: https://test.pypi.org/simple/, https://pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 4.3 MB/s  0:00:00m eta 0:00:01
  Attempting uninstall: syngen
    Found existing installation: syngen 0.10.33rc1
    Uninstalling syngen-0.10.33rc1:
      Successfully uninstalled syngen-0.10.33rc1


# Class initialization

```python
Syngen(
    metadata_path: Optional[str] = None,  # use a metadata file in the '.yaml', '.yml' format for a training or an inference of one or multiple tables to centralize all parameters in one place
    table_name: Optional[str] = None,     # required for a single-table training or inference process; an arbitrary string used to name the directories where artifacts are stored
    source: Optional[str] = None,         # optional for a single-table training or inference process; a path to the file that you want to use as a reference
    loader: Optional[Callable[[str], pd.DataFrame]] # optional for a training or inference process of one or multiple tables; a callback function that returns the sample of the original data of a certain table
)
```
### Attributes description:

- **`metadata_path`** *(Optional[str], default: None)*: a path to a metadata file in *'.yaml'* or *'.yml'* format, used during a training or inference processes to centralize all parameters for one or multiple tables in one place.
- **`table_name`** *(Optional[str], default: None)*: a required parameter for a training or inference processes of a single table; an arbitrary string used to name the directories where artifacts are stored.
- **`source`** *(Optional[str], default: None)*: an optional parameter for a single-table training or inference process; a path to the file that you want to use as a reference.
- **`loader`** *(Optional[str], default: None)*: an optional parameter for a training or inference process of one or multiple tables; a callback function that returns the sample of the original data of a certain table.

***Note***: You can provide the information about required attributes in one of three ways:
1. Use `metadata_path` to define the metadata of one or multiple tables and their relationships in one place.
2. Using `table_name` and `source`: For a single-table training or inference process, provide both `table_name` and `source` if the original data is supplied directly from a `source`.
3. Using `table_name` and `loader`: For a single-table training or inference process, provide both `table_name` and `loader` if the original data is supplied via a callback function (`loader`).
4. Using `metadata_path` and `loader`: For a training or inference process of one or multiple tables if the original data of tables is supplied via a callback function (`loader`).

In [2]:
# The example of the initialization of the instance of the class 'Syngen' for the single-table training or inference process
# by providing the `source` and 'table_name' parameters
from syngen.sdk import Syngen


launcher_for_single_table_with_source = Syngen(
    source="../examples/example-data/housing.csv", 
    table_name="housing"
)

2026-01-08 11:38:55.901777: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-01-08 11:38:55.901838: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-01-08 11:38:56.049737: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# The example of the initialization of the instance of the class 'Syngen' for the single-table training or inference process 
# by providing the `loader` and 'table_name' parameters
import os
import pandas as pd

from syngen.sdk import Syngen



def get_dataframe(table_name: str) -> pd.DataFrame:
    """
    The demonstration callback function to load a CSV file 
    and get a pandas DataFrame from the 'example-data' directory.
    
    Parameters
    ----------
    table_name : str
        The name of the table (the name of the file without extension) to load
    
    Returns
    -------
    pd.DataFrame
        The loaded data as a pandas DataFrame
    """
    path_to_example_data = f"../examples/example-data/{table_name}.csv"
    
    if not os.path.exists(path_to_example_data):
        raise FileNotFoundError(f"The CSV file '{table_name}.csv' does not exist")
    
    return pd.read_csv(path_to_example_data)



launcher_for_single_table_with_loader = Syngen(
    loader=get_dataframe, 
    table_name="housing"
)

In [4]:
# The example of the initialization of the instance of the class 'Syngen' for multiple tables
from syngen.sdk import Syngen


launcher_for_multiple_tables = Syngen(
    metadata_path="../examples/example-metadata/housing_metadata.yaml"
)

# Launch training

You can start a training process using the SDK entrypoint `Syngen().train(...)`. This will train a model and save the model artifacts to a disk in the directory *'./model_artifacts'*. The SDK mirrors the CLI options so you can pass the same parameters programmatically. Below is a complete description of all available parameters:

```python
train(
    self,
    epochs: int = 10,                     # a number of training epochs
    drop_null: bool = False,              # whether to drop rows with at least one missing value
    row_limit: Optional[int] = None,      # a number of rows to train over
    reports: Union[str, Tuple[str], List[str]] = "none",  # report types: "none", "accuracy", "sample", "metrics_only", "all"
    log_level: Literal["TRACE", "DEBUG", "INFO", "WARNING", "ERROR", "CRITICAL"] = "INFO", # a logging level
    batch_size: int = 32,                 # a training batch size
    fernet_key: Optional[str] = None      # a name of the environment variable containing the Fernet key for secure storage of the data subset
)
```

### Parameters description:

- **`epochs`** *(int, default: 10)*: A number of training epochs. Must be ≥ 1. Since the early stopping mechanism is implemented the bigger value of epochs is the better.

- **`drop_null`** *(bool, default: False)*: Whether to drop rows containing at least one missing value before training. When `False`, missing values are handled during the training process.

- **`row_limit`** *(Optional[int], default: None)*: A maximum number of rows to use for training. If specified and less than the total rows, a random subset of the specified size will be selected. Useful for testing or working with large datasets.

- **`reports`** *(Union[str, Tuple[str], List[str]], default: "none")*: Controls generation of quality reports. Accepts single string or list of strings:
  - `"none"` - no reports generated (default)
  - `"accuracy"` - generates an accuracy report comparing synthetic data (same size as original) with original dataset to estimate the quality of training process
  - `"sample"` - generates a sample report showing distribution comparisons between the original data and the subset of this data
  - `"metrics_only"` - outputs metrics to stdout without generation of an accuracy report
  - `"all"` - generates both accuracy and sample reports

  List example: `["accuracy", "sample"]` to generate multiple report types

  *Note*: Report generation may require significant time for large tables (>10,000 rows)

- **`log_level`** *(str, default: "INFO")*: A logging level for the training process. Accepted values: `"TRACE"`, `"DEBUG"`, `"INFO"`, `"WARNING"`, `"ERROR"`, `"CRITICAL"`.

- **`batch_size`** *(int, default: 32)*: A training batch size. Must be ≥ 1. Splits training into batches to optimize memory usage. Smaller batches use less RAM but may increase training time.

- **`fernet_key`** *(Optional[str], default: None)*: A name of the environment variable containing a 44-character URL-safe base64-encoded Fernet key. When provided, the data subset is encrypted on a disk (stored in the `.dat` format). If not provided, data is stored unencrypted in the `.pkl` format. **Important**: The same key must be used during an inference and a report generation to decrypt the data.


*Note:* For full documentation, metadata file format, and additional details, please refer to [README.md](../README.md)

In [5]:
# The example of a training the single table on the provided data fetching it via the 'source' parameter


launcher_for_single_table_with_source.train(
    epochs=5,
    drop_null=False,
    row_limit=1000, 
    batch_size=100,
    reports="all",
    log_level="DEBUG"
)

2026-01-08 11:45:40.276 | DEBUG    | syngen.ml.validation_schema.validation_schema:validate_schema:348 - The schema of the metadata is valid
2026-01-08 11:45:40.291 | INFO     | syngen.ml.config.validation:_collect_errors:508 - The validation of the metadata has been passed successfully
2026-01-08 11:45:40.291 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:check_mlflow_server:32 - MLFlow server URL not provided
2026-01-08 11:45:40.291 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:create_tracker:69 - MLFlow server is either unreachable or not set up, therefore the tracking will not be performed
2026-01-08 11:45:40.291 | INFO     | syngen.train:launch_train:104 - The training process will be executed according to the information mentioned in 'train_settings' in the metadata file. If appropriate information is absent from the metadata file, then the values of parameters sent through CLI will be used. Otherwise, the values of parameters will be defaulted.
2026-01-08 11:45:40.3

32/32 [==============================] - 0s 865us/step


2026-01-08 11:45:45.966 | INFO     | syngen.ml.vae.models.model:fit_sampler:187 - Creating BayesianGaussianMixture
2026-01-08 11:45:45.966 | INFO     | syngen.ml.vae.models.model:fit_sampler:189 - Fitting BayesianGaussianMixture
2026-01-08 11:45:48.008 | INFO     | syngen.ml.vae.models.model:fit_sampler:191 - Finished fitting BayesianGaussianMixture
2026-01-08 11:45:48.053 | INFO     | syngen.ml.vae.wrappers.wrappers:save_state:545 - Saved VAE state in model_artifacts/resources/housing/vae/checkpoints
2026-01-08 11:45:48.053 | INFO     | syngen.ml.handlers.handlers:__fit_model:191 - Finished VAE training
2026-01-08 11:45:48.053 | INFO     | syngen.ml.handlers.handlers:handle:143 - No columns to train kde over found
2026-01-08 11:45:48.053 | INFO     | syngen.ml.strategies.strategies:run:153 - Training of the table - 'housing' was completed
2026-01-08 11:45:48.055 | INFO     | syngen.ml.worker.worker:_infer_table:519 - Infer process of the table - 'housing' has started


 1/32 [..............................] - ETA: 4s

2026-01-08 11:45:48.471 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing/vae/checkpoints
2026-01-08 11:45:48.471 | DEBUG    | syngen.ml.handlers.handlers:handle:490 - Infer model with parameters: size=1000, run_parallel=False, batch_size=1000, random_seed=1
2026-01-08 11:45:48.471 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 1 batch(es)
2026-01-08 11:45:48.471 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 1 of 1
2026-01-08 11:45:48.471 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-08 11:45:48.472 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing' started.


32/32 [==============================] - 0s 1ms/step


Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 7985.00it/s]
2026-01-08 11:45:48.725 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2026-01-08 11:45:48.730 | INFO     | syngen.ml.strategies.strategies:run:236 - Synthesis of the table - 'housing' was completed. Synthetic data saved in 'model_artifacts/tmp_store/housing/merged_infer_housing.csv'
2026-01-08 11:45:48.730 | WARNING  | syngen.ml.reporters.reporters:generate_report:241 - The report(s) generation might be time-consuming
2026-01-08 11:45:48.730 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:287 - The calculation of sample metrics for the table - 'housing' has started
2026-01-08 11:45:51.839 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:287 - The sample report of the table - 'housing' has been generated
2026-01-08 11:45:51.839 | INFO     | syngen.ml.reporters.reporters:_log_and_update_pro

In [6]:
launcher_for_single_table_with_source.execution_artifacts

{'housing': {'losses_path': 'model_artifacts/system_store/losses/losses-housing-2026-01-08-11-45-40-376493.csv',
  'path_to_input_data': 'model_artifacts/tmp_store/housing/input_data_housing.pkl',
  'generated_reports': {'sample_report': 'model_artifacts/resources/housing/reports/sample-report-2026_01_08_11_45_51_837973.html',
   'accuracy_report': 'model_artifacts/resources/housing/reports/accuracy-report-2026_01_08_11_46_11_837175.html'}}}

In [7]:
# The example of training the single table on the provided data fetching it via the 'loader' parameter


launcher_for_single_table_with_loader.train(
    epochs=5,
    drop_null=False,
    row_limit=1000, 
    batch_size=100, 
    reports="all",
    log_level="DEBUG"
)

2026-01-08 11:47:50.051 | DEBUG    | syngen.ml.validation_schema.validation_schema:validate_schema:348 - The schema of the metadata is valid
2026-01-08 11:47:50.055 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:103 - The artifacts located in the path - 'model_artifacts/resources/housing/' was removed
2026-01-08 11:47:50.077 | INFO     | syngen.ml.data_loaders.dataframe_fetcher:fetch_data:21 - Successfully fetched dataframe for table: 'housing'
2026-01-08 11:47:50.077 | INFO     | syngen.ml.config.validation:_collect_errors:508 - The validation of the metadata has been passed successfully
2026-01-08 11:47:50.078 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:check_mlflow_server:32 - MLFlow server URL not provided
2026-01-08 11:47:50.078 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:create_tracker:69 - MLFlow server is either unreachable or not set up, therefore the tracking will not be performed
2026-01-08 11:47:50.078 | INFO     | syngen.train:launch_train:

32/32 [==============================] - 0s 835us/step


2026-01-08 11:47:55.265 | INFO     | syngen.ml.vae.wrappers.wrappers:_train:410 - epoch: 4, total loss: 5.728543758392334, time: 0.9237 sec
2026-01-08 11:47:55.268 | INFO     | syngen.ml.vae.models.model:fit_sampler:179 - Fit sampler
2026-01-08 11:47:55.276 | INFO     | syngen.ml.vae.models.model:fit_sampler:183 - Start encoding
2026-01-08 11:47:55.424 | INFO     | syngen.ml.vae.models.model:fit_sampler:187 - Creating BayesianGaussianMixture
2026-01-08 11:47:55.424 | INFO     | syngen.ml.vae.models.model:fit_sampler:189 - Fitting BayesianGaussianMixture
2026-01-08 11:47:58.021 | INFO     | syngen.ml.vae.models.model:fit_sampler:191 - Finished fitting BayesianGaussianMixture
2026-01-08 11:47:58.061 | INFO     | syngen.ml.vae.wrappers.wrappers:save_state:545 - Saved VAE state in model_artifacts/resources/housing/vae/checkpoints
2026-01-08 11:47:58.061 | INFO     | syngen.ml.handlers.handlers:__fit_model:191 - Finished VAE training
2026-01-08 11:47:58.061 | INFO     | syngen.ml.handlers.h

32/32 [==============================] - 0s 1ms/step


2026-01-08 11:47:58.479 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing/vae/checkpoints
2026-01-08 11:47:58.479 | DEBUG    | syngen.ml.handlers.handlers:handle:490 - Infer model with parameters: size=1000, run_parallel=False, batch_size=1000, random_seed=1
2026-01-08 11:47:58.479 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 1 batch(es)
2026-01-08 11:47:58.479 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 1 of 1
2026-01-08 11:47:58.479 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-08 11:47:58.480 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing' started.
Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 7806.66it/s]
2026-01-08 11:47:58.725 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%)

In [8]:
launcher_for_single_table_with_loader.execution_artifacts

{'housing': {'losses_path': 'model_artifacts/system_store/losses/losses-housing-2026-01-08-11-47-50-095845.csv',
  'path_to_input_data': 'model_artifacts/tmp_store/housing/input_data_housing.pkl',
  'generated_reports': {'sample_report': 'model_artifacts/resources/housing/reports/sample-report-2026_01_08_11_48_02_153098.html',
   'accuracy_report': 'model_artifacts/resources/housing/reports/accuracy-report-2026_01_08_11_48_26_696270.html'}}}

In [9]:
# The example of training of multiple tables with relationships


launcher_for_multiple_tables.train(log_level="DEBUG")

2026-01-08 11:51:18.073 | DEBUG    | syngen.ml.validation_schema.validation_schema:validate_schema:348 - The schema of the metadata is valid
2026-01-08 11:51:18.077 | INFO     | syngen.ml.config.validation:_collect_errors:508 - The validation of the metadata has been passed successfully
2026-01-08 11:51:18.078 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:check_mlflow_server:32 - MLFlow server URL not provided
2026-01-08 11:51:18.078 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:create_tracker:69 - MLFlow server is either unreachable or not set up, therefore the tracking will not be performed
2026-01-08 11:51:18.078 | INFO     | syngen.train:launch_train:104 - The training process will be executed according to the information mentioned in 'train_settings' in the metadata file. If appropriate information is absent from the metadata file, then the values of parameters sent through CLI will be used. Otherwise, the values of parameters will be defaulted.
2026-01-08 11:51:18.0

25/25 [==============================] - 0s 967us/step


2026-01-08 11:51:31.321 | INFO     | syngen.ml.vae.models.model:fit_sampler:187 - Creating BayesianGaussianMixture
2026-01-08 11:51:31.321 | INFO     | syngen.ml.vae.models.model:fit_sampler:189 - Fitting BayesianGaussianMixture
2026-01-08 11:51:31.876 | INFO     | syngen.ml.vae.models.model:fit_sampler:191 - Finished fitting BayesianGaussianMixture
2026-01-08 11:51:31.910 | INFO     | syngen.ml.vae.wrappers.wrappers:save_state:545 - Saved VAE state in model_artifacts/resources/housing-properties/vae/checkpoints
2026-01-08 11:51:31.911 | INFO     | syngen.ml.handlers.handlers:__fit_model:191 - Finished VAE training
2026-01-08 11:51:31.911 | INFO     | syngen.ml.handlers.handlers:handle:143 - No columns to train kde over found
2026-01-08 11:51:31.911 | INFO     | syngen.ml.strategies.strategies:run:153 - Training of the table - 'housing_properties' was completed
2026-01-08 11:51:31.920 | INFO     | syngen.ml.processors.processors:_preprocess_data:121 - As the parameter 'drop_null' set t

57/57 [==============================] - 0s 744us/step


2026-01-08 11:51:45.327 | INFO     | syngen.ml.vae.models.model:fit_sampler:187 - Creating BayesianGaussianMixture
2026-01-08 11:51:45.327 | INFO     | syngen.ml.vae.models.model:fit_sampler:189 - Fitting BayesianGaussianMixture
2026-01-08 11:51:46.590 | INFO     | syngen.ml.vae.models.model:fit_sampler:191 - Finished fitting BayesianGaussianMixture
2026-01-08 11:51:46.617 | INFO     | syngen.ml.vae.wrappers.wrappers:save_state:545 - Saved VAE state in model_artifacts/resources/housing-conditions/vae/checkpoints
2026-01-08 11:51:46.617 | INFO     | syngen.ml.handlers.handlers:__fit_model:191 - Finished VAE training
2026-01-08 11:51:46.617 | INFO     | syngen.ml.handlers.handlers:handle:143 - No columns to train kde over found
2026-01-08 11:51:46.617 | INFO     | syngen.ml.strategies.strategies:run:153 - Training of the table - 'housing_conditions' was completed
2026-01-08 11:51:46.620 | INFO     | syngen.ml.worker.worker:_infer_table:519 - Infer process of the table - 'housing_properti

...

2026-01-08 11:51:46.926 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing-properties/vae/checkpoints
2026-01-08 11:51:46.927 | DEBUG    | syngen.ml.handlers.handlers:handle:490 - Infer model with parameters: size=790, run_parallel=False, batch_size=790, random_seed=1
2026-01-08 11:51:46.927 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 1 batch(es)
2026-01-08 11:51:46.927 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_properties'. Generating the batch 1 of 1
2026-01-08 11:51:46.927 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-08 11:51:46.927 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_properties' started.
Generation of the data...: 100%|██████████| 7/7 [00:00<00:00, 7329.04it/s]
2026-01-08 11:51:47.098 | INFO     | syngen.ml.strategies.strategies:run:236 - Synthesis of the table 

32/32 [==============================] - 0s 1ms/step


2026-01-08 11:51:47.300 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing-conditions/vae/checkpoints
2026-01-08 11:51:47.301 | DEBUG    | syngen.ml.handlers.handlers:handle:490 - Infer model with parameters: size=1799, run_parallel=False, batch_size=1000, random_seed=1
2026-01-08 11:51:47.301 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 2 batch(es)
2026-01-08 11:51:47.301 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_conditions'. Generating the batch 1 of 2
2026-01-08 11:51:47.301 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-08 11:51:47.301 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_conditions' started.
Generation of the data...: 100%|██████████| 3/3 [00:00<00:00, 4061.62it/s]

25/25 [==============================] - 0s 964us/step



2026-01-08 11:51:47.434 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_conditions'. Generating the batch 2 of 2
2026-01-08 11:51:47.434 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-08 11:51:47.434 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_conditions' started.
Generation of the data...: 100%|██████████| 3/3 [00:00<00:00, 4808.14it/s]
2026-01-08 11:51:47.514 | INFO     | syngen.ml.handlers.handlers:generate_keys:428 - The 'households' assigned as a foreign_key feature
2026-01-08 11:51:47.539 | INFO     | syngen.ml.strategies.strategies:run:236 - Synthesis of the table - 'housing_conditions' was completed. Synthetic data saved in 'model_artifacts/tmp_store/housing-conditions/merged_infer_housing-conditions.csv'
2026-01-08 11:51:47.539 | WARNING  | syngen.ml.reporters.reporters:generate_report:241 - The report(s) generation might be time-consuming
2026-01-08 11:5

The *'execution_artifacts'* attribute of the **Syngen** class provides information about the generated artifacts during the training process:

In [10]:
from pprint import pprint

pprint(launcher_for_multiple_tables.execution_artifacts)

{'housing_conditions': {'generated_reports': {'accuracy_report': 'model_artifacts/resources/housing-conditions/reports/accuracy-report-2026_01_08_11_51_52_285756.html',
                                              'sample_report': 'model_artifacts/resources/housing-conditions/reports/sample-report-2026_01_08_11_51_48_326116.html'},
                        'losses_path': 'model_artifacts/system_store/losses/losses-housing-conditions-2026-01-08-11-51-31-922263.csv',
                        'path_to_input_data': 'model_artifacts/tmp_store/housing-conditions/input_data_housing-conditions.pkl'},
 'housing_properties': {'generated_reports': {'accuracy_report': 'model_artifacts/resources/housing-properties/reports/accuracy-report-2026_01_08_11_52_01_227608.html',
                                              'sample_report': 'model_artifacts/resources/housing-properties/reports/sample-report-2026_01_08_11_51_53_404560.html'},
                        'losses_path': 'model_artifacts/system_sto

# Launch generation of synthetic data

You can start an inference process using the SDK entrypoint `Syngen().infer(...)`. The SDK mirrors the CLI options so you can pass the same parameters programmatically. Below is a complete description of all available parameters:

```python
infer(
    self,
    size: int = 100,                      # the desired number of rows to generate
    run_parallel: bool = False,           # whether to use multiprocessing (feasible for tables > 50000 rows)
    batch_size: Optional[int] = None,     # an inference batch size
    random_seed: Optional[int] = None,    # if specified, generates a reproducible result
    reports: Union[str, List[str]] = "none",  # report types: "none", "accuracy", "metrics_only", "all"
    log_level: str = "INFO",              # a logging level
    fernet_key: Optional[str] = None      # a name of the environment variable containing the Fernet key for decrypting the data subset
)
```
### Parameters description:

- **`size`** *(int, default: 100)*: The desired number of synthetic rows to generate. Must be ≥ 1.

- **`run_parallel`** *(bool, default: False)*: Whether to use multiprocessing for data generation. Set to `True` to enable parallel processing, which is recommended and feasible for generating large tables (> 50000 rows).

- **`batch_size`** *(Optional[int], default: None)*: The inference batch size. Must be ≥ 1. If specified, the generation is split into batches to optimize memory usage and save RAM.

- **`random_seed`** *(Optional[int], default: None)*: A random seed for reproducible generation. Must be ≥ 0.

- **`reports`** *(Union[str, Tuple[str], List[str]], default: "none")*: Controls generation of quality reports. Accepts single string or list of strings:
  - `"none"` - no reports generated (default)
  - `"accuracy"` - generates an accuracy report comparing original and synthetic data patterns to verify quality of a generated data
  - `"metrics_only"` - outputs metrics to stdout without generating an accuracy report
  - `"all"` - generates an accuracy report (same as `"accuracy"`)

  List example: `["accuracy", "metrics_only"]` to generate multiple report types

  *Note*: Report generation may require significant time for large generated tables (>10,000 rows)

- **`log_level`** *(str, default: "INFO")*: A logging level for the inference process. Accepted values: `"TRACE"`, `"DEBUG"`, `"INFO"`, `"WARNING"`, `"ERROR"`, `"CRITICAL"`.

- **`fernet_key`** *(Optional[str], default: None)*: A name of the environment variable containing a 44-character URL-safe base64-encoded Fernet key. When provided, the data subset is decrypted for a report generation. **Important**: The same key used during a training must be used during a report generation to successfully decrypt the data.


*Note:* For full documentation, metadata file format, and additional details, please refer to [README.md](../README.md)

In [12]:
# The example of inference for the single table

launcher_for_single_table_with_source.infer(
    size=100000,
    run_parallel=False,
    batch_size=5000,
    random_seed=42,
    reports="all",
    log_level="DEBUG"
)

2026-01-08 12:03:43.195 | DEBUG    | syngen.ml.validation_schema.validation_schema:validate_schema:348 - The schema of the metadata is valid
2026-01-08 12:03:43.196 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:103 - The artifacts located in the path - 'model_artifacts/tmp_store/housing/merged_infer_housing.csv' was removed
2026-01-08 12:03:43.196 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:103 - The artifacts located in the path - 'model_artifacts/tmp_store/housing/infer_message.success' was removed
2026-01-08 12:03:43.197 | WARNING  | syngen.ml.config.validation:_check_existence_of_destination:229 - As the destination path wasn't specified for the table - 'housing', the synthetic data will be stored at the default path - './model_artifacts/tmp_store/housing/merged_infer_housing.csv'
2026-01-08 12:03:43.197 | INFO     | syngen.ml.config.validation:_collect_errors:508 - The validation of the metadata has been passed successfully
2026-01-08 12:03:43.197

  1/157 [..............................] - ETA: 20s

2026-01-08 12:03:43.619 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing/vae/checkpoints
2026-01-08 12:03:43.619 | DEBUG    | syngen.ml.handlers.handlers:handle:490 - Infer model with parameters: size=100000, run_parallel=False, batch_size=5000, random_seed=42, reports - 'accuracy'
2026-01-08 12:03:43.620 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 20 batch(es)
2026-01-08 12:03:43.620 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 1 of 20
2026-01-08 12:03:43.620 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-08 12:03:43.620 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing' started.


157/157 [==============================] - 0s 1ms/step


Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 2959.04it/s]

...

Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 3752.53it/s]

 95/157 [=================>............] - ETA: 0s


2026-01-08 12:03:51.693 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2026-01-08 12:03:51.694 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 20 of 20
2026-01-08 12:03:51.694 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-08 12:03:51.694 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing' started.


157/157 [==============================] - 0s 1ms/step


Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 3713.27it/s]
2026-01-08 12:03:52.105 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2026-01-08 12:03:52.440 | INFO     | syngen.ml.strategies.strategies:run:236 - Synthesis of the table - 'housing' was completed. Synthetic data saved in 'model_artifacts/tmp_store/housing/merged_infer_housing.csv'
2026-01-08 12:03:52.441 | WARNING  | syngen.ml.reporters.reporters:generate_report:241 - The report(s) generation might be time-consuming
2026-01-08 12:03:52.441 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:287 - The calculation of accuracy metrics for the table - 'housing' has started
2026-01-08 12:03:53.798 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:193 - Median accuracy is 0.8228
Generating bivariate distributions...: 100%|██████████| 45/45 [00:40<00:00,  1.10it/s]
2026-01-08 12:04:39.250 | I

In [14]:
# The example of inference for the single table

launcher_for_single_table_with_loader.infer(
    size=100000,
    run_parallel=False,
    batch_size=5000,
    random_seed=42,
    reports="all",
    log_level="DEBUG"
)

2026-01-08 12:05:18.460 | DEBUG    | syngen.ml.validation_schema.validation_schema:validate_schema:348 - The schema of the metadata is valid
2026-01-08 12:05:18.462 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:103 - The artifacts located in the path - 'model_artifacts/tmp_store/housing/merged_infer_housing.csv' was removed
2026-01-08 12:05:18.462 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:103 - The artifacts located in the path - 'model_artifacts/tmp_store/housing/infer_message.success' was removed
2026-01-08 12:05:18.462 | WARNING  | syngen.ml.config.validation:_check_existence_of_destination:229 - As the destination path wasn't specified for the table - 'housing', the synthetic data will be stored at the default path - './model_artifacts/tmp_store/housing/merged_infer_housing.csv'
2026-01-08 12:05:18.463 | INFO     | syngen.ml.config.validation:_collect_errors:508 - The validation of the metadata has been passed successfully
2026-01-08 12:05:18.463

  1/157 [..............................] - ETA: 19s

2026-01-08 12:05:18.891 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing/vae/checkpoints
2026-01-08 12:05:18.891 | DEBUG    | syngen.ml.handlers.handlers:handle:490 - Infer model with parameters: size=100000, run_parallel=False, batch_size=5000, random_seed=42, reports - 'accuracy'
2026-01-08 12:05:18.891 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 20 batch(es)
2026-01-08 12:05:18.891 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 1 of 20
2026-01-08 12:05:18.891 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-08 12:05:18.892 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing' started.


157/157 [==============================] - 0s 1ms/step


Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 3096.88it/s]

...

Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 3808.28it/s]

 90/157 [================>.............] - ETA: 0s


2026-01-08 12:05:27.013 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2026-01-08 12:05:27.013 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 20 of 20
2026-01-08 12:05:27.013 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-08 12:05:27.014 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing' started.


157/157 [==============================] - 0s 1ms/step


Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 3753.75it/s]
2026-01-08 12:05:27.434 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2026-01-08 12:05:27.766 | INFO     | syngen.ml.strategies.strategies:run:236 - Synthesis of the table - 'housing' was completed. Synthetic data saved in 'model_artifacts/tmp_store/housing/merged_infer_housing.csv'
2026-01-08 12:05:27.767 | WARNING  | syngen.ml.reporters.reporters:generate_report:241 - The report(s) generation might be time-consuming
2026-01-08 12:05:27.767 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:287 - The calculation of accuracy metrics for the table - 'housing' has started
2026-01-08 12:05:29.144 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:193 - Median accuracy is 0.8228
Generating bivariate distributions...: 100%|██████████| 45/45 [00:40<00:00,  1.11it/s]
2026-01-08 12:06:13.784 | I

In [15]:
# The example of inference of multiple tables with relationships


launcher_for_multiple_tables.infer(log_level="DEBUG")

2026-01-08 12:11:57.530 | DEBUG    | syngen.ml.validation_schema.validation_schema:validate_schema:348 - The schema of the metadata is valid
2026-01-08 12:11:57.531 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:103 - The artifacts located in the path - 'model_artifacts/tmp_store/housing-properties/merged_infer_housing-properties.csv' was removed
2026-01-08 12:11:57.531 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:103 - The artifacts located in the path - 'model_artifacts/tmp_store/housing-properties/infer_message.success' was removed
2026-01-08 12:11:57.532 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:103 - The artifacts located in the path - 'model_artifacts/tmp_store/housing-conditions/merged_infer_housing-conditions.csv' was removed
2026-01-08 12:11:57.532 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:103 - The artifacts located in the path - 'model_artifacts/tmp_store/housing-conditions/infer_message.success' was rem

3/3 [==============================] - 0s 2ms/step


2026-01-08 12:11:57.856 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing-properties/vae/checkpoints
2026-01-08 12:11:57.856 | DEBUG    | syngen.ml.handlers.handlers:handle:490 - Infer model with parameters: size=90, run_parallel=False, batch_size=90, random_seed=10, reports - 'accuracy'
2026-01-08 12:11:57.857 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 1 batch(es)
2026-01-08 12:11:57.857 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_properties'. Generating the batch 1 of 1
2026-01-08 12:11:57.857 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-08 12:11:57.857 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_properties' started.
Generation of the data...: 100%|██████████| 7/7 [00:00<00:00, 11410.85it/s]
2026-01-08 12:11:57.995 | INFO     | syngen.ml.strategies.strategies:run:236 - S

3/3 [==============================] - 0s 1ms/step


2026-01-08 12:11:58.199 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing-conditions/vae/checkpoints
2026-01-08 12:11:58.199 | DEBUG    | syngen.ml.handlers.handlers:handle:490 - Infer model with parameters: size=90, run_parallel=False, batch_size=90, random_seed=10, reports - 'accuracy'
2026-01-08 12:11:58.199 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 1 batch(es)
2026-01-08 12:11:58.200 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_conditions'. Generating the batch 1 of 1
2026-01-08 12:11:58.200 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-08 12:11:58.200 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_conditions' started.
Generation of the data...: 100%|██████████| 3/3 [00:00<00:00, 7458.75it/s]
2026-01-08 12:11:58.302 | INFO     | syngen.ml.handlers.handlers:generate_keys:42

The *'execution_artifacts'* attribute of the **Syngen** class provides information about the generated artifacts during the inference process:

In [16]:
pprint(launcher_for_multiple_tables.execution_artifacts)

{'housing_conditions': {'generated_reports': {'accuracy_report': 'model_artifacts/tmp_store/housing-conditions/reports/accuracy-report-2026_01_08_12_12_01_818895.html'},
                        'path_to_generated_data': 'model_artifacts/tmp_store/housing-conditions/merged_infer_housing-conditions.csv',
                        'path_to_input_data': 'model_artifacts/tmp_store/housing-conditions/input_data_housing-conditions.pkl'},
 'housing_properties': {'generated_reports': {'accuracy_report': 'model_artifacts/tmp_store/housing-properties/reports/accuracy-report-2026_01_08_12_12_09_455652.html'},
                        'path_to_generated_data': 'model_artifacts/tmp_store/housing-properties/merged_infer_housing-properties.csv',
                        'path_to_input_data': 'model_artifacts/tmp_store/housing-properties/input_data_housing-properties.pkl'}}


# Data security: Using Fernet key for encryption

In the current implementation, a sample of the original data is stored on a disk during a training process. To ensure data security and protect sensitive information, you can use a **Fernet key** to encrypt this data.

## What is a Fernet key?

A Fernet key is a 44-character URL-safe base64-encoded string used for symmetric encryption. When provided, the data subset is encrypted on a disk (stored in the `.dat` format instead of unencrypted `.pkl` format).

## How to generate a Fernet key

You can generate a Fernet key using the following code:

In [17]:
# Generate a Fernet key
from cryptography.fernet import Fernet

fernet_key = Fernet.generate_key().decode("utf-8")

## Setting the Fernet Key as an environment variable

After generating the key, you need to store it as an environment variable. This can be done in your terminal or programmatically in Python.

### Option 1: Set in Terminal (Linux/macOS)

```bash
export MY_FERNET_KEY='your_generated_fernet_key_here'
```

### Option 2: Set in Terminal (Windows)

```cmd
set MY_FERNET_KEY=your_generated_fernet_key_here
```

### Option 3: Set programmatically in Python

```python
import os
os.environ['MY_FERNET_KEY'] = 'your_generated_fernet_key_here'
```

## Using the Fernet Key in a training

When training with encryption, pass the name of the environment variable (not the Fernet key itself) to the `fernet_key` parameter:

In [31]:
# The example: the training with the Fernet key encryption

import os
from syngen.sdk import Syngen

# Step 1: Set the Fernet key as an environment variable
os.environ["MY_FERNET_KEY"] = fernet_key  # Using the key generated above

# Step 2: Train with encryption enabled

launcher_for_encrypted_data = Syngen(
    source="../examples/example-data/housing.csv", 
    table_name="housing_encrypted"
)

launcher_for_encrypted_data.train(
    epochs=5,
    row_limit=1000, 
    batch_size=32, 
    fernet_key="MY_FERNET_KEY"  # Pass the environment variable name, not the key itself
)

2026-01-08 12:30:17.268 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:103 - The artifacts located in the path - 'model_artifacts/resources/housing-encrypted/' was removed
2026-01-08 12:30:17.271 | WARNING  | syngen.ml.config.validation:_launch_validation:468 - Encryption and decryption are enabled for the table 'housing_encrypted' as a Fernet key is provided
2026-01-08 12:30:17.271 | INFO     | syngen.ml.config.validation:_collect_errors:508 - The validation of the metadata has been passed successfully
2026-01-08 12:30:17.271 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:check_mlflow_server:32 - MLFlow server URL not provided
2026-01-08 12:30:17.271 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:create_tracker:69 - MLFlow server is either unreachable or not set up, therefore the tracking will not be performed
2026-01-08 12:30:17.271 | INFO     | syngen.train:launch_train:104 - The training process will be executed according to the information mentioned in '

32/32 [==============================] - 0s 670us/step


2026-01-08 12:30:31.323 | INFO     | syngen.ml.vae.models.model:fit_sampler:187 - Creating BayesianGaussianMixture
2026-01-08 12:30:31.324 | INFO     | syngen.ml.vae.models.model:fit_sampler:189 - Fitting BayesianGaussianMixture
2026-01-08 12:30:33.420 | INFO     | syngen.ml.vae.models.model:fit_sampler:191 - Finished fitting BayesianGaussianMixture
2026-01-08 12:30:33.460 | INFO     | syngen.ml.vae.wrappers.wrappers:save_state:545 - Saved VAE state in model_artifacts/resources/housing-encrypted/vae/checkpoints
2026-01-08 12:30:33.460 | INFO     | syngen.ml.handlers.handlers:__fit_model:191 - Finished VAE training
2026-01-08 12:30:33.460 | INFO     | syngen.ml.handlers.handlers:handle:143 - No columns to train kde over found
2026-01-08 12:30:33.460 | INFO     | syngen.ml.strategies.strategies:run:153 - Training of the table - 'housing_encrypted' was completed
2026-01-08 12:30:33.462 | INFO     | syngen.ml.data_loaders.data_loaders:save_data:681 - Data is successfully encrypted and save

In [19]:
launcher_for_encrypted_data.execution_artifacts

{'housing_encrypted': {'losses_path': 'model_artifacts/system_store/losses/losses-housing-encrypted-2026-01-08-12-14-06-933918.csv',
  'path_to_input_data': 'model_artifacts/tmp_store/housing-encrypted/input_data_housing-encrypted.dat',
  'generated_reports': {}}}

## Using the Fernet Key in an inference

**Important**: When generating synthetic data during an inference process, you must use the **same Fernet key** that was used during a training process. This allows the system to decrypt the stored data subset for a report generation.

If the Fernet key is not provided or doesn't match the Fernet key used in the training process, the inference process will fail when trying to access the encrypted data.

In [32]:
# Example: Inference with Fernet key decryption
# The environment variable 'MY_FERNET_KEY' is already set from the training step

# Inference with the same Fernet key


launcher_for_encrypted_data.infer(
    size=100000,
    batch_size=5000, 
    random_seed=42,
    reports="all",
    fernet_key="MY_FERNET_KEY"  # Must use the same key as in a training
)

2026-01-08 12:30:40.204 | WARNING  | syngen.ml.config.validation:_launch_validation:468 - Encryption and decryption are enabled for the table 'housing_encrypted' as a Fernet key is provided
2026-01-08 12:30:40.205 | WARNING  | syngen.ml.config.validation:_check_existence_of_destination:229 - As the destination path wasn't specified for the table - 'housing_encrypted', the synthetic data will be stored at the default path - './model_artifacts/tmp_store/housing-encrypted/merged_infer_housing-encrypted.csv'
2026-01-08 12:30:40.207 | INFO     | syngen.ml.data_loaders.data_loaders:load_data:706 - Data stored at the path - 'model_artifacts/tmp_store/housing-encrypted/input_data_housing-encrypted.dat' has been successfully decrypted and loaded.
2026-01-08 12:30:40.207 | INFO     | syngen.ml.config.validation:_collect_errors:508 - The validation of the metadata has been passed successfully
2026-01-08 12:30:40.207 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:check_mlflow_server:32 - MLF

  1/157 [..............................] - ETA: 20s

2026-01-08 12:30:40.627 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing-encrypted/vae/checkpoints
2026-01-08 12:30:40.627 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 20 batch(es)
2026-01-08 12:30:40.627 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_encrypted'. Generating the batch 1 of 20
2026-01-08 12:30:40.627 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-08 12:30:40.628 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_encrypted' started.


157/157 [==============================] - 0s 1ms/step


Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 5556.71it/s]

...

Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 5482.75it/s]

130/157 [=======================>......] - ETA: 0s


2026-01-08 12:30:48.595 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2026-01-08 12:30:48.596 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_encrypted'. Generating the batch 20 of 20
2026-01-08 12:30:48.596 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-08 12:30:48.596 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_encrypted' started.


157/157 [==============================] - 0s 1ms/step


Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 5469.75it/s]
2026-01-08 12:30:49.014 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2026-01-08 12:30:49.350 | INFO     | syngen.ml.strategies.strategies:run:236 - Synthesis of the table - 'housing_encrypted' was completed. Synthetic data saved in 'model_artifacts/tmp_store/housing-encrypted/merged_infer_housing-encrypted.csv'
2026-01-08 12:30:49.351 | WARNING  | syngen.ml.reporters.reporters:generate_report:241 - The report(s) generation might be time-consuming
2026-01-08 12:30:49.351 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:287 - The calculation of accuracy metrics for the table - 'housing_encrypted' has started
2026-01-08 12:30:49.352 | INFO     | syngen.ml.data_loaders.data_loaders:load_data:706 - Data stored at the path - 'model_artifacts/tmp_store/housing-encrypted/input_data_housing-encrypted.dat' has been s

In [33]:
launcher_for_encrypted_data.execution_artifacts

{'housing_encrypted': {'path_to_input_data': 'model_artifacts/tmp_store/housing-encrypted/input_data_housing-encrypted.dat',
  'path_to_generated_data': 'model_artifacts/tmp_store/housing-encrypted/merged_infer_housing-encrypted.csv',
  'generated_reports': {'accuracy_report': 'model_artifacts/tmp_store/housing-encrypted/reports/accuracy-report-2026_01_08_12_31_15_192801.html'}}}

## Using the Fernet Key with the metadata file

You can also specify the Fernet key in the metadata file for both training and inference:

```yaml
global:
  encryption:
    fernet_key: MY_FERNET_KEY  # Name of the environment variable

TABLE_NAME:
  train_settings:
    source: "./data/table.csv"
  
  infer_settings:
    size: 100
  
  # You can also specify per-table encryption
  encryption:
    fernet_key: MY_FERNET_KEY
```

Then use it in your code:

```python
# Training with the metadata file and the Fernet key
Syngen(metadata_path="path/to/metadata.yaml").train()

# Inference with the metadata file and the Fernet key
Syngen(metadata_path="path/to/metadata.yaml").infer()
```

## Important security notes

⚠️ **Critical security considerations:**

1. **Store the key securely**: Never hardcode the Fernet key directly in your code or commit it to version control systems.
2. **Key recovery is impossible**: If you lose the Fernet key, encrypted data cannot be recovered
3. **Same key required**: Always use the same Fernet key for a training, an inference and a report generation
4. **Environment variable**: Use an environment variable to store the Fernet key securely
5. **Key length**: The Fernet key must be exactly 44 characters (URL-safe base64-encoded)
6. **Production environments**: In production, use secure secret management services (AWS Secrets Manager, Azure Key Vault, HashiCorp Vault, etc.)

## What happens without a Fernet key?

If you don't provide a `fernet_key` parameter:
- Data subset is stored **unencrypted** in the `.pkl` format
- No decryption is needed during the inference or the report generation
- Suitable for non-sensitive data or development environments

With a `fernet_key`:
- Data subset is stored **encrypted** in the `.dat` format
- Decryption is required during the inference or the report generation using the same Fernet key
- Recommended for sensitive or production data

# Generate quality reports separately

Sometimes you may want to generate quality reports separately after training or/and inference has already been completed to evaluate the quality of the input data or the generated data. The SDK provides the `Syngen().generate_quality_reports(...)` method that allows you to generate quality reports for a table using existing artifacts without re-running the training or/and inference processes.

This method is useful when:
- You completed training/inference without quality reports (with `reports="none"`)
- You want to generate additional report types later
- You want to separate the computation-intensive training/inference from report generation


```python
generate_quality_reports(
    self,
    table_name: str,                        # required: the name of the table to generate quality reports for
    reports: Union[str, Tuple[str], List[str]], # required: report types to generate
    fernet_key: Optional[str] = None        # optional: a Fernet key for decrypting encrypted data
)
```

### Parameters description:

- **`table_name`** *(str, required)*: The name of the table to generate reports for.

- **`reports`** *(Union[str, Tuple[str], List[str]], required)*: Controls which quality reports to generate. Accepts single string or list of strings:
  - `"accuracy"` - generates an accuracy report comparing original and synthetic data
  - `"metrics_only"` - outputs metrics information to stdout without generating an accuracy report
  - `"sample"` - generates a sample report showing distribution comparisons between original data and the data subset used for a training process
  - `"all"` - generates all available reports (*"accuracy"* and *"sample"*)
  
  List example: `["accuracy", "sample"]` to generate multiple report types.

  *Note*: Report generation may require significant time for large tables (>10,000 rows)

- **`fernet_key`** *(Optional[str], default: None)*: The name of the environment variable containing the Fernet key used to decrypt the original data subset. **Important**: Must be the same key used during training if the data was encrypted.

### Required artifacts

To generate quality reports, the following artifacts must exist:

**For accuracy reports (`"accuracy"` or `"metrics_only"`):**
- Training must be completed successfully
- Inference must be completed successfully

**For a `"sample"` report:**
- Training must be completed successfully

### Key notes:

- The method uses existing artifacts and does not re-run a training or an inference process
- All required artifacts must be present in the `model_artifacts` directory
- The `table_name` must match exactly the name of the `table_name` that was used in a training/inference process
- If data was encrypted during a training, the same `fernet_key` must be provided

*Note:* For full documentation and additional details, please refer to [README.md](../README.md)

In [20]:
# Example 1: Generate an accuracy report after a training and inference process completed without a generation of quality reports.
# Assume a training and inference process were already completed with reports="none".
# Now generate an accuracy report separately.


launcher_for_multiple_tables.generate_quality_reports(
    table_name="housing_conditions",
    reports="accuracy",
    log_level="DEBUG"
)

2026-01-08 12:25:03.344 | WARNING  | syngen.ml.reporters.reporters:generate_report:241 - The report(s) generation might be time-consuming
2026-01-08 12:25:03.345 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:287 - The calculation of accuracy metrics for the table - 'housing_conditions' has started
2026-01-08 12:25:03.497 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:193 - Median accuracy is 0.8178
Generating bivariate distributions...: 100%|██████████| 3/3 [00:01<00:00,  1.73it/s]
2026-01-08 12:25:05.979 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:221 - Median of differences of correlations is 0.2862
2026-01-08 12:25:06.117 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:229 - Mean clusters homogeneity is 0.1556
2026-01-08 12:25:06.118 | INFO     | syngen.ml.metrics.metrics_classes.metrics:calculate_all:1289 - Calculating utility metric
Calculating utility metric for multiclass cla

In [21]:
launcher_for_multiple_tables.execution_artifacts

{'housing_conditions': {'generated_reports': {'accuracy_report': 'model_artifacts/tmp_store/housing-conditions/reports/accuracy-report-2026_01_08_12_25_07_370096.html'}}}

In [22]:
# Example 2: Generate a sample report after training completed

# Generate a sample report to compare original data with its subset
launcher_for_multiple_tables.generate_quality_reports(
    table_name="housing_properties",
    reports="sample"
)

2026-01-08 12:26:23.724 | WARNING  | syngen.ml.reporters.reporters:generate_report:241 - The report(s) generation might be time-consuming
2026-01-08 12:26:23.724 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:287 - The calculation of sample metrics for the table - 'housing_properties' has started
2026-01-08 12:26:24.843 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:287 - The sample report of the table - 'housing_properties' has been generated


In [23]:
launcher_for_multiple_tables.execution_artifacts

{'housing_properties': {'generated_reports': {'sample_report': 'model_artifacts/resources/housing-properties/reports/sample-report-2026_01_08_12_26_24_842237.html'}}}

In [ ]:
# Example 3: Generate multiple reports at once

# Generate both accuracy and sample reports
launcher_for_multiple_tables.generate_quality_reports(
    table_name="housing_conditions",
    reports=["accuracy", "sample"]
)

# Or use "all" to generate all available reports
launcher_for_multiple_tables.generate_quality_reports(
    table_name="housing_conditions",
    reports="all"
)

2026-01-08 12:29:18.415 | WARNING  | syngen.ml.reporters.reporters:generate_report:241 - The report(s) generation might be time-consuming
2026-01-08 12:29:18.416 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:287 - The calculation of accuracy metrics for the table - 'housing_conditions' has started
2026-01-08 12:29:18.559 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:193 - Median accuracy is 0.8178
Generating bivariate distributions...: 100%|██████████| 3/3 [00:01<00:00,  1.77it/s]
2026-01-08 12:29:21.004 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:221 - Median of differences of correlations is 0.2862
2026-01-08 12:29:21.144 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:229 - Mean clusters homogeneity is 0.1556
2026-01-08 12:29:21.144 | INFO     | syngen.ml.metrics.metrics_classes.metrics:calculate_all:1289 - Calculating utility metric
Calculating utility metric for multiclass cla

In [29]:
launcher_for_multiple_tables.execution_artifacts

{'housing_conditions': {'generated_reports': {'accuracy_report': 'model_artifacts/tmp_store/housing-conditions/reports/accuracy-report-2026_01_08_12_29_21_842755.html',
   'sample_report': 'model_artifacts/resources/housing-conditions/reports/sample-report-2026_01_08_12_29_22_575420.html'}}}

In [34]:
# Example 4: Generate quality reports for encrypted data
import os

# Ensure the Fernet key environment variable is set
# (Should be the same key used during training)
os.environ['MY_FERNET_KEY'] = fernet_key

# Generate qulaity reports with decryption
launcher_for_encrypted_data.generate_quality_reports(
    table_name="housing_encrypted",
    reports="accuracy",
    fernet_key="MY_FERNET_KEY"  # Same key used in training
)

2026-01-08 12:39:49.224 | WARNING  | syngen.ml.reporters.reporters:generate_report:241 - The report(s) generation might be time-consuming
2026-01-08 12:39:49.224 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:287 - The calculation of accuracy metrics for the table - 'housing_encrypted' has started
2026-01-08 12:39:49.226 | INFO     | syngen.ml.data_loaders.data_loaders:load_data:706 - Data stored at the path - 'model_artifacts/tmp_store/housing-encrypted/input_data_housing-encrypted.dat' has been successfully decrypted and loaded.
2026-01-08 12:39:50.373 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:193 - Median accuracy is 0.8458
Generating bivariate distributions...: 100%|██████████| 45/45 [00:18<00:00,  2.50it/s]
2026-01-08 12:40:12.514 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:221 - Median of differences of correlations is 0.2903
2026-01-08 12:40:13.397 | INFO     | syngen.ml.metrics.accuracy_test.accu

In [35]:
launcher_for_encrypted_data.execution_artifacts

{'housing_encrypted': {'generated_reports': {'accuracy_report': 'model_artifacts/tmp_store/housing-encrypted/reports/accuracy-report-2026_01_08_12_40_15_136981.html'}}}

In [36]:
# Example 5: Generate the "metrics_only" report (without a full accuracy report)
# Output metrics to stdout


launcher_for_multiple_tables.generate_quality_reports(
    table_name="housing",
    reports="metrics_only"
)

2026-01-08 12:40:56.134 | WARNING  | syngen.ml.reporters.reporters:generate_report:241 - The report(s) generation might be time-consuming
2026-01-08 12:40:56.134 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:287 - The calculation of accuracy metrics for the table - 'housing' has started
2026-01-08 12:40:57.360 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:193 - Median accuracy is 0.8228
2026-01-08 12:40:57.466 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:221 - Median of differences of correlations is 0.0047
2026-01-08 12:40:58.275 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:229 - Mean clusters homogeneity is 0.0009
2026-01-08 12:40:58.276 | INFO     | syngen.ml.metrics.metrics_classes.metrics:calculate_all:1289 - Calculating utility metric
Calculating utility metric for multiclass classification: 100%|██████████| 2/2 [00:00<00:00,  5.45it/s]
Calculating utility metric for binary

In [38]:
launcher_for_single_table_with_loader.generate_quality_reports(
    table_name="housing",
    reports="sample"
)

2026-01-08 12:41:35.666 | WARNING  | syngen.ml.reporters.reporters:generate_report:241 - The report(s) generation might be time-consuming
2026-01-08 12:41:35.666 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:287 - The calculation of sample metrics for the table - 'housing' has started
2026-01-08 12:41:35.678 | INFO     | syngen.ml.data_loaders.dataframe_fetcher:fetch_data:21 - Successfully fetched dataframe for table: 'housing'
2026-01-08 12:41:35.678 | WARNING  | syngen.ml.reporters.reporters:_fetch_dataframe:64 - The original data of the table - 'housing' has been fetched using the callback function. The data may have been modified since the beginning of the training process.
2026-01-08 12:41:38.339 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:287 - The sample report of the table - 'housing' has been generated


# Using custom data loaders

By providing the callback function to the `loader` allows you to load data from non-standard sources such as databases, APIs, Cloud storage, or any custom data source. This is particularly useful when:
- Your data is stored in a database (PostgreSQL, MySQL, MongoDB, etc.)
- You need to fetch data from an API or web service
- Your data is in Cloud storage (AWS S3, Azure Blob Storage, Google Cloud Storage, etc.)
- You need to apply custom preprocessing before a training/inference process
- The file format is not directly supported

## The requirements to the custom data loader

A custom callback function provided to `loader` must meet these requirements:

1. **Accept `table_name` as the first parameter** (str): The name of the table to load
2. **Return a pandas DataFrame**: The loaded data must be returned as a pandas DataFrame

## The callback function signature

```python
def custom_loader(table_name: str) -> pd.DataFrame:
    """
    Load data for the specified table.
    
    Parameters
    ----------
    table_name : str
        The name of the table to load
    
    Returns
    -------
    pd.DataFrame
        The loaded data as a pandas DataFrame
    """
    # Your loading logic here
    return dataframe
```

## The example: the basic custom data loader
```python
from pathlib import Path
from typing import Optional
import pandas as pd


def get_dataframe(table_name: str) -> pd.DataFrame:
    """
    The demonstration of the callback function to load a CSV file,  
    and get a pandas DataFrame from the 'example-data' directory.
    
    Parameters
    ----------
    table_name : str
        The name of the table (the name of the file without extension) to load
    
    Returns
    -------
    pd.DataFrame
        The loaded data as a pandas DataFrame
    """
    path_to_example_data = f"../examples/example-data/{table_name}.csv"
    
    if not os.path.exists(path_to_example_data):
        raise FileNotFoundError(f"The CSV file '{table_name}.csv' does not exist")
    
    return pd.read_csv(path_to_example_data)
```

## How to use the custom data loader

### Step 1: Ensure the custom data loader function is importable

Make sure your custom data loader function is importable.

### Step 2: Provide the custom data loader function during the initialization of the class 'Syngen'

```python
from syngen.sdk import Syngen

launcher = Syngen(loader=get_dataframe, table_name="housing")

launcher.train(
    epochs=5,
    row_limit=1000,
    batch_size=32
)

launcher.infer(
    size=10000,
    batch_size=5000,
    reports="accuracy"
)
```



# Data loading and saving: DataIO class

The SDK provides the `DataIO` class for loading and saving data in various supported formats with optional encryption and format settings. This class is useful when you need to:
- Load and save data in different file formats (*CSV*, *Avro*, *Excel*, etc.)
- Load and save data with specific format settings
- Work with encrypted data files

## Class initialization

```python
DataIO(
    path: str,                          # required: a path to the data file
    fernet_key: Optional[str] = None,   # optional: a Fernet key for encrypted data
    **kwargs                            # optional: format settings for CSV or Excel tables, or schema for AVRO file
)
```

### Parameters description:

- **`path`** *(str, required)*: the path to the data file to load or save. Supported formats include:
  - CSV files: `.csv`, `.psv`, `.tsv`, `.txt`
  - Avro files: `.avro`
  - Excel files: `.xls`, `.xlsx`

- **`fernet_key`** *(Optional[str], default: None)*: the name of the environment variable containing the Fernet key for encrypted data operations.

- **`**kwargs`**: Optional format settings or/and a schema for reading and writing data. Available parameters depend on the file format:

  **For tables in '.csv', '.psv', '.tsv', '.txt' formats:**
  - `sep` *(str)*: Delimiter to use (e.g., `','`, `';'`, `'\t'`)
  - `quotechar` *(str)*: Character used to denote the start and end of a quoted item (default: `'"'`)
  - `quoting` *(str)*: Quoting behavior - `"all"`, `"minimal"`, `"non-numeric"`, `"none"`
  - `escapechar` *(str)*: Character used to escape other characters
  - `encoding` *(str)*: Encoding to use (e.g., `'utf-8'`, `'latin-1'`)
  - `header` *(Optional[int, List[int], Literal["infer"]])*: Row number(s) containing column labels and marking the start of the data
  - `skiprows` *(Optional[int, List[int]])*: Lines to skip at the start of the file
  - `on_bad_lines` *(Literal["error", "warn", "skip"])*: Action on bad lines - `"error"`, `"warn"`, `"skip"`
  - `engine` *(Optional[Literal["c", "python"]])*: Parser engine - `"c"`, `"python"`
  - `na_values` *(Opional[List[str]])*: Additional strings to recognize as NA/NaN

  **For Excel formats (.xls, .xlsx):**
  - `sheet_name` *(Optional[str, int, List[Union[int, str]])*: Name or index of the sheet to read

### Available methods

`load_data(**kwargs)`

Loads data from the specified file path and returns it as a pandas DataFrame.

`load_schema()`

Returns the original schema of the loaded data, including column names and data types. Available only for data in the *'.avro'* format.

`save_data(df, **kwargs)`

Saves a pandas DataFrame to the specified file path with(without) the configured format settings or schema.

*Note:* For full documentation and additional details, please refer to [README.md](../README.md)

In [ ]:
# The example 1: Load CSV data with default settings

from syngen.sdk import DataIO

data_io = DataIO(path="../examples/example-data/housing.csv")

df = data_io.load_data()

print(f"Loaded {df.shape[0]} rows and {df.shape[1]} columns")
print(f"\nFirst few rows:")
print(df.head())

In [ ]:
# The example 2: Load CSV data with custom format settings

from syngen.sdk import DataIO

data_io = DataIO(
    path="../examples/example-data/escaped_quoted_table.csv",
    sep=',',           # delimiter
    quotechar='"',     # quote character
    quoting="minimal", # quoting style
    encoding='utf-8',  # encoding
    header=0           # use first row as header
)

df = data_io.load_data()

print(f"Loaded {df.shape[0]} rows and {df.shape[1]} columns")
print(f"\nFirst few rows:")
print(df.head())

In [ ]:
# The example 3: Load data and get schema information

from syngen.sdk import DataIO


data_io = DataIO(path="../examples/example-data/avro_file.avro")

df = data_io.load_data()

schema = data_io.load_schema()

print("Data schema:")
print(schema)

In [ ]:
# The example 4: Save data to a file

import pandas as pd
from syngen.sdk import DataIO


sample_data = pd.DataFrame({
    'id': [1, 2, 3, 4, 5],
    'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'age': [25, 30, 35, 40, 45],
    'city': ['New York', 'London', 'Paris', 'Tokyo', 'Sydney']
})


data_io = DataIO(
    path="../examples/example-data/sample_output.csv",
    sep=',',
    encoding='utf-8'
)


data_io.save_data(sample_data)

print("Data saved successfully to '../examples/example-data/sample_output.csv'")